**Laser Embedding**

In [1]:
!pip install torch==2.2.1

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [2]:
!pip install -q laserembeddings==1.1.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.8/859.8 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 880.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 33.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 1.13.1 which is incompatible.
torchdata 0.7.1 requires torch>=2, but you have torch 1.13.1 which is incompatible.
torchtext 0.17.1 requires torch

In [3]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [4]:
import pandas as pd
import numpy as np
import os

In [5]:
#datapath = os.path.join('..', 'data', 'preprocessed_messages.csv')
#dataframe = pd.read_csv(datapath)
dataframe = pd.read_csv('preprocessed_messages.csv')

In [6]:
texts = dataframe['text']
tokenized = dataframe['stemmed_sentence_Porter']

In [7]:
langs = ["en" for i in range(len(texts))]

In [8]:
%%time

from laserembeddings import Laser

laser = Laser()
embeddings_text = laser.embed_sentences(
    texts,
    lang=langs)

embeddings_tokenized = laser.embed_sentences(
    tokenized,
    lang=langs)

CPU times: user 6.17 s, sys: 1.34 s, total: 7.51 s
Wall time: 7.93 s


In [9]:
print('Text embedding shape: ', embeddings_text.shape)
print('Tokenized embedding shape: ', embeddings_tokenized.shape)

Text embedding shape:  (50, 1024)
Tokenized embedding shape:  (50, 1024)


In [10]:
print('text:', embeddings_text[0], texts[0])
print('embeddings:', embeddings_tokenized[0], texts[0], tokenized[0])

text: [ 0.0086212  -0.0001213   0.04254882 ...  0.01014943  0.00585767
  0.0261275 ] We can do it online so that everyone could join
embeddings: [1.4702949e-02 5.2515816e-06 1.8564619e-02 ... 2.1697991e-02 2.4591812e-03
 2.8372105e-02] We can do it online so that everyone could join ['onlin', 'everyon', 'could', 'join']


In [11]:
reference = 'let s do the meeting'
reference_tokenized =  ['do', 'meeting']
embedding_reference = laser.embed_sentences(
    reference,
    lang='en')
print('text:', embedding_reference)

text: [[ 0.01682114  0.01596329  0.00836292 ...  0.00719668  0.01053112
  -0.00772411]]


In [12]:
from sklearn.metrics.pairwise import cosine_similarity
similarities_text = cosine_similarity(embedding_reference, embeddings_text)
most_similar_text = np.argmax(similarities_text)

In [13]:
most_similar_indices_text = np.argsort(similarities_text[0])[:-11:-1]
print(texts[most_similar_indices_text])

45                                       I vote the 4th
1                             I'm ok if we do it online
22                                 I manage to add them
23                                        I can try now
49         We could use our names/surnames concatenated
0       We can do it online so that everyone could join
8     I coudn't do it directly when saving the messa...
27    I pushed the new version of test (just with th...
17    It's probably something with the working direc...
21    wow,\r\ngreat\r\nCan you send the csv, I just ...
Name: text, dtype: object


In [14]:
similarities_tokenized = cosine_similarity(embedding_reference, embeddings_tokenized)
most_similar_tokenized = np.argmax(similarities_tokenized)

In [15]:
most_similar_indices_token = np.argsort(similarities_tokenized[0])[:-11:-1]
print(texts[most_similar_indices_token], tokenized[most_similar_indices_token])

45                                  I vote the 4th
49    We could use our names/surnames concatenated
22                            I manage to add them
23                                   I can try now
16                                         Perfect
29                                             Yes
9                                         perfetto
38                               I think it’s this
28                                           great
25                                   ahh, too late
Name: text, dtype: object 45                                 ['vote', '4th']
49    ['could', 'use', 'names/surnam', 'concaten']
22                                ['manag', 'add']
23                                         ['tri']
16                                     ['perfect']
29                                          ['ye']
9                                     ['perfetto']
38                                  ['think', '’']
28                                       ['great']
25   

In [16]:
print('Most similar index:\ntext: ', most_similar_text, '\ntokenized: ', most_similar_tokenized)

Most similar index:
text:  45 
tokenized:  45


In [17]:
print('Most similar text:', texts[most_similar_text])
print('Most similar tokenized:', texts[most_similar_tokenized], tokenized[most_similar_tokenized])

Most similar text: I vote the 4th
Most similar tokenized: I vote the 4th ['vote', '4th']



------------------------------------

**Sentence Transformers**

In [18]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 2.4 MB/s eta 0:00:00


Check https://huggingface.co/sentence-transformers for lower dimensionality transformers

In [19]:
from sentence_transformers import SentenceTransformer

emodel = SentenceTransformer("average_word_embeddings_glove.6B.300d") #300

embedded_texts = emodel.encode(texts)
embedded_tokenized = emodel.encode(tokenized)
reference_embedding =  emodel.encode(reference)
reference_embedding_tokenized =  emodel.encode(reference_tokenized)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/248 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

(…)beddings/whitespacetokenizer_config.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

(…)WordEmbeddings/wordembedding_config.json:   0%|          | 0.00/164 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/480M [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [20]:
print(reference_embedding.shape, embedded_texts.shape, embedded_tokenized.shape)

(300,) (50, 300) (50, 300)


In [21]:
similarities_text = cosine_similarity(reference_embedding.reshape(1, -1), embedded_texts)
most_similar_text = np.argmax(similarities_text)

In [23]:
similarities_text

array([[ 0.44043884,  0.37137914,  0.4379762 ,  0.45897532,  0.17536844,
         0.3251886 ,  0.6264845 ,  0.42395973,  0.2736226 , -0.18987867,
         0.48635527,  0.6173249 ,  0.50872123,  0.43110955,  0.40224394,
         0.4595701 ,  0.31389052,  0.4543349 ,  0.5268014 ,  0.41694683,
         0.50474244,  0.48124766,  0.42730498,  0.56646794,  0.42395973,
         0.19872993,  0.45971087,  0.39712468,  0.35586435,  0.3777862 ,
         0.660203  ,  0.6997585 ,  0.5426404 ,  0.49928492,  0.4973064 ,
         0.52030593,  0.8074545 ,  0.56949115,  0.570318  ,  0.0609519 ,
         0.3967919 ,  0.        ,  0.        ,  0.        ,  0.44951025,
         0.383224  ,  0.        ,  0.53217006,  0.48896378,  0.13609406]],
      dtype=float32)

In [26]:
#Given a window of message, print 'yes' only if the reference message is related to at least one message in the window
if any(score > 0.7 for score in similarities_text.flatten()):
  print("yes")
else:
  print("no")

yes


In [27]:
most_similar_indices = np.argsort(similarities_text[0])[:-11:-1]
print(texts[most_similar_indices])

36    I asked them where they want to meet, I'll let...
31                 Did you meet the polinetwork’s guys?
30    Yeah,\r\nIt's probably ok. He was very support...
6     Do you want to meet/call sometime soon or we w...
11    If you react with different reactions to this ...
38                                    I think it’s this
37                        Is tomorrow at 14 ok for you?
23                                        I can try now
32    hmm\r\nare you sure? did you change the values...
47    It would be more convenient to set it related ...
Name: text, dtype: object


In [35]:
similarities_token = cosine_similarity(reference_embedding.reshape(1, -1), embedded_tokenized)
most_similar_token = np.argmax(similarities_token)

In [36]:
if any(score > 0.7 for score in similarities_token.flatten()):
  print("yes")
else:
  print("no")

yes


In [33]:
most_similar_indices_token = np.argsort(similarities_token[0])[:-11:-1]
print(texts[most_similar_indices_token], tokenized[most_similar_indices_token])

36    I asked them where they want to meet, I'll let...
6     Do you want to meet/call sometime soon or we w...
30    Yeah,\r\nIt's probably ok. He was very support...
31                 Did you meet the polinetwork’s guys?
33    Now it works just fine, I don't know why it di...
26    Also, if you want to commit the changes in the...
17    It's probably something with the working direc...
47    It would be more convenient to set it related ...
37                        Is tomorrow at 14 ok for you?
13    I guess the csv can be modified once transform...
Name: text, dtype: object 36    ['ask', 'want', 'meet', ',', "'ll", 'let', 'kn...
6     ['want', 'meet/cal', 'sometim', 'soon', 'work'...
30    ['yeah', ',', "'s", 'probabl', 'ok', '.', 'sup...
31             ['meet', 'polinetwork', '’', 'guy', '?']
33    ['work', 'fine', ',', "n't", 'know', "n't", 'w...
26    ['also', ',', 'want', 'commit', 'chang', 'repo...
17    ["'s", 'probabl', 'someth', 'work', 'directori...
47    ['would', 'conve

In [37]:
print('Most similar index.\nText:', most_similar_text, '\nTokenized:', most_similar_token)

Most similar index.
Text: 36 
Tokenized: 36


In [38]:
print('Most similar text.\nText:', texts[most_similar_text], '\nTokenized:', texts[most_similar_token], tokenized[most_similar_token])

Most similar text.
Text: I asked them where they want to meet, I'll let you know as soon as I receive an answer 
Tokenized: I asked them where they want to meet, I'll let you know as soon as I receive an answer ['ask', 'want', 'meet', ',', "'ll", 'let', 'know', 'soon', 'receiv', 'answer']


Try with the 'standard' messages

In [41]:
messages = ["We can do it online so that everyone could join",
         'I am ok if we do it online',
         "I'm free but I prefer online",
         "I am free from 11:30 to 15:30"]

emodel = SentenceTransformer("average_word_embeddings_glove.6B.300d")  # Using GloVe embeddings

embedded_texts = emodel.encode(messages)

last_message = "I am free from 11:30 to 15:30"
embedded_last_message = emodel.encode([last_message])[0]

is_correlated = False

for embedded_text in embedded_texts[:-1]:
    similarity = cosine_similarity([embedded_last_message], [embedded_text])[0][0]

    if similarity >= 0.7:
        is_correlated = True
        break

if is_correlated:
    print("Yes")
else:
    print("No")


No


In [ ]:
# other options
# https://huggingface.co/spaces/mteb/leaderboard

Apply PCA

*PCA on original text and reference*

In [42]:
from sklearn.decomposition import PCA

all_texts = reference + texts
all_embeddings = emodel.encode(all_texts)

# Apply PCA to the encoded embeddings
pca = PCA(n_components=50)
pca.fit(all_embeddings)
embedded_texts_pca = pca.transform(all_embeddings)

reference_embedding_pca = embedded_texts_pca[0]
embedded_texts_pca = embedded_texts_pca[1:]


In [43]:
similarities_text = cosine_similarity(reference_embedding_pca.reshape(1, -1), embedded_texts_pca)
most_similar_text = np.argmax(similarities_text)
most_similar_indices = np.argsort(similarities_text[0])[:-11:-1]
print(texts[most_similar_indices])

1                             I'm ok if we do it online
0       We can do it online so that everyone could join
43                                 amidierfan@gmail.com
5     Thursday, I'm all available except from 5:15 t...
34    @Martinavigano \r\nTell me if you could find t...
16                                              Perfect
9                                              perfetto
46    TelegramInfromationSpace\r\nAnamolyDetectionBo...
20    When I run directly from the terminal I get th...
25                                        ahh, too late
Name: text, dtype: object


*PCA on tokenized text and original reference (think about tokenizing the reference...)*

In [44]:
all_texts = reference + tokenized
all_embeddings = emodel.encode(all_texts)

# Apply PCA to the encoded embeddings
pca = PCA(n_components=50)
pca.fit(all_embeddings)
embedded_tokenized_pca = pca.transform(all_embeddings)

reference_embedding_pca = embedded_tokenized_pca[0]
embedded_tokenized_pca = embedded_tokenized_pca[1:]

In [45]:
similarities_tokenized = cosine_similarity(reference_embedding_pca.reshape(1, -1), embedded_tokenized_pca)
most_similar_tokenized = np.argmax(similarities_tokenized)
most_similar_indices = np.argsort(similarities_tokenized[0])[:-11:-1]
print(texts[most_similar_indices])

27    I pushed the new version of test (just with th...
42                       Martina2.vigano@mail.polimi.it
41                         lorenzo.mondo@mail.polimi.it
40    Do you remember the name of the technique prop...
28                                                great
15            Could you do it like reactions: {"🤞": 1}?
22                                 I manage to add them
45                                       I vote the 4th
8     I coudn't do it directly when saving the messa...
35                        Room 2.1.3, it should be free
Name: text, dtype: object


-------------------------------

**Roberta**

In [46]:
import torch
from transformers import RobertaTokenizer, RobertaModel

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaModel.from_pretrained('roberta-base')

'''
With this model, cosine similarity seems to decrease (test it with a broader dataset, alternative to Roberta)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')
'''

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


"\nWith this model, cosine similarity seems to decrease (test it with a broader dataset, alternative to Roberta)\ntokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')\nmodel = DistilBertModel.from_pretrained('distilbert-base-uncased')\n"

In [47]:
model.eval()

messages = ["We can do it online so that everyone could join",
            'I am ok if we do it online',
            "I'm free but I prefer online",
            "I am free from 11:30 to 15:30"]

input_last = tokenizer(messages[-1], return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    output_last = model(**input_last) # Embedding for the last message

embedding_last = output_last.last_hidden_state[:, 0, :]

In [48]:
is_correlated = False

for i in range(len(messages)-1):

    input_prev = tokenizer(messages[i], return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        output_prev = model(**input_prev)

    embedding_prev = output_prev.last_hidden_state[:, 0, :]

    similarity = cosine_similarity(embedding_last, embedding_prev)[0][0]
    print(similarity)
    if similarity >= 0.7:
        is_correlated = True
        break

if is_correlated:
    print("Yes")
else:
    print("No")

0.99836904
Yes


Try and see the result with the texts in the dataframe

In [52]:
model.eval()

messages = [texts, reference]

input_last = tokenizer(messages[-1], return_tensors="pt", padding=True, truncation=True)

with torch.no_grad():
    output_last = model(**input_last) # Embedding for the last message

embedding_last = output_last.last_hidden_state[:, 0, :]

In [57]:
is_correlated = False

for i in range(len(messages)-1):
    input_prev = tokenizer(messages[0][i], return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        output_prev = model(**input_prev)

    embedding_prev = output_prev.last_hidden_state[:, 0, :]

    similarity = cosine_similarity(embedding_last, embedding_prev)[0][0]
    print(similarity)
    if similarity >= 0.7:
        is_correlated = True
        break

if is_correlated:
    print("Yes")
else:
    print("No")

0.9986888
Yes
